In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
import smtplib
from email.message import EmailMessage
import datetime

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import smtplib
from email.message import EmailMessage
import datetime

days_counter=1
start_date='2022-08-23'
today=''
while today!=str(datetime.date.today()+datetime.timedelta(days=1)):

    today=str(datetime.date(2022,8,23)+datetime.timedelta(days=days_counter))
    
    email_id='sai.kartikeya05@gmai.com' #Enter your Gmail ID
    sender_name='Myself' #Enter your name
    app_password='yfygskcaozqsthun' #Generate App password and paste here


    tickers=['RELIANCE.NS','ADANIENT.NS','LT.NS','AXISBANK.NS','ITC.NS']

    quantities={'RELIANCE.NS':4,
                'ADANIENT.NS':3,
                'LT.NS':6,
                'AXISBANK.NS':14,
                'ITC.NS':31}

    buy_avg={'RELIANCE.NS':2366.45,
             'ADANIENT.NS':3030.65,
             'LT.NS':1877.05,
             'AXISBANK.NS':747.05,
             'ITC.NS':315.90}

    invested_value=[]
    final_df={}

    for ticker in tickers:

        stock_df=yf.download(ticker,start=start_date,end=today)
        stock_df['Change']=stock_df['Adj Close'].diff()
        stock_df['%Change']=stock_df['Adj Close'].pct_change()*100
        sub_df=stock_df.iloc[-1]
        final_df[ticker]=sub_df.to_dict()
        invested_value.append((quantities[ticker]*buy_avg[ticker]))

    portfolio_df=pd.DataFrame(final_df).T
    portfolio_df.index.name='Tickers'
    portfolio_df['Quantity']=quantities
    portfolio_df['Invested Amount']=invested_value
    portfolio_df['Market Value']=portfolio_df['Close']*portfolio_df['Quantity']
    portfolio_df['Days P&L']=portfolio_df['Change']*portfolio_df['Quantity']
    portfolio_df['All time P&L']=portfolio_df['Market Value']-portfolio_df['Invested Amount']
    portfolio_df['All time P&L(%)']=(((portfolio_df['Market Value']-portfolio_df['Invested Amount'])/portfolio_df['Invested Amount'])*100)
    portfolio_df=portfolio_df.round(2)

    fund_df=pd.DataFrame()
    fund_df.index=[stock_df.index[-1]]
    fund_df['Invested Amount']=[portfolio_df['Invested Amount'].sum()]
    fund_df['Market Value']=[portfolio_df['Market Value'].sum()]
    fund_df['Days P&L']=[portfolio_df['Days P&L'].sum()]
    fund_df['Daily percent change']=((fund_df['Days P&L'])/(fund_df['Market Value']-fund_df['Days P&L']))*100
    fund_df['All Time percent change']=((fund_df['Market Value']-fund_df['Invested Amount'])/fund_df['Invested Amount'])*100

    excel_df=pd.read_excel('FundAutomation.xlsx',index_col=0)

    def service(send_email:bool):
        concat_df=pd.concat([excel_df,fund_df],ignore_index=False)
        writer = pd.ExcelWriter('FundAutomation.xlsx', engine='xlsxwriter')
        concat_df.to_excel(writer,sheet_name='Portfolio Movement',index=True)
        writer.close()
        
        if send_email:
            msg = EmailMessage()
            msg['Subject'] = f'Ticker Summary for {today}'
            msg['From'] = f"{sender_name} <{email_id}>"
            msg['To'] = f"{email_id}"
            msg.add_alternative(portfolio_df.to_html(index=True), subtype='html')

            with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
                smtp.login(email_id, app_password) 
                smtp.send_message(msg)

    if len(excel_df.columns)==0:
        service(send_email=False)
    elif excel_df.index[-1]==fund_df.index:
        pass
    else:
        service(send_email=False)
    days_counter+=1
    print(f'Done for {today}')






In [7]:
portfolio_df

,Open,High,Low,Close,Adj Close,Volume,Change,%Change,Quantity,Invested Amount,Market Value,Days P&L,All time P&L,All time P&L(%)
Tickers,,,,,,,,,,,,,,
RELIANCE.NS,2524.20,2542.85,2516.95,2522.20,2522.20,4754427.0,2.80,0.11,4,9465.80,10088.80,11.20,623.00,6.58
ADANIENT.NS,2720.00,2722.40,2488.85,2530.30,2530.30,5247693.0,-167.75,-6.22,3,9091.95,7590.90,-503.25,-1501.05,-16.51
LT.NS,2689.55,2726.00,2682.45,2717.60,2717.60,1989300.0,38.80,1.45,6,11262.30,16305.60,232.80,5043.30,44.78
AXISBANK.NS,962.00,980.45,958.40,979.25,979.25,12676090.0,22.20,2.32,14,10458.70,13709.50,310.80,3250.80,31.08
ITC.NS,455.60,455.65,448.60,450.45,450.45,8443150.0,-3.80,-0.84,31,9792.90,13963.95,-117.80,4171.05,42.59


In [24]:
start_date='2023-08-11'
today=str(datetime.date(2023,8,11)+datetime.timedelta(days=3))
today

'2023-08-14'

In [4]:
import pandas as pd

writer = pd.ExcelWriter('FundAutomation.xlsx', engine='xlsxwriter')
reset=pd.DataFrame()
reset.to_excel(writer,sheet_name='Sheet1')
writer.close()
